En el siguiente ejercicio, se buscará obtener e integrar la información sobre el tipo de cambio proporcionada por el Banco Central Europeo (ECB)

# Web Scraping

Primero, es necesario obtener las monedas manejadas por el ECB desde el siguiente link: https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html"

In [3]:
response_html =
#verificar que se obtivo una respuesta exitosa
response_html.status_code, response_html.reason

(200, 'OK')

In [4]:
#obtener el texto dentro de la respuesta
htmltext = 
htmltext[:100]

'\n<!DOCTYPE html>\n<html lang="en">\n\t<head>\r\n    <meta charset="utf-8">\r\n\t<meta http-equiv="X-UA-Compa'

In [5]:
#convertirlo a un objeto manejable
htmltext = htmltext.replace('\n', '').replace('\t', '').replace('\r', '')
soupObj = 

In [6]:
#Come se observa dentro de la estructura de la pégina web, las clases que nos interesan 
#dentro de cada fila son "currency" y "alignLeft"
currency_codes = soupObj.findAll('td', {"class":"currency"})
currency_names = soupObj.findAll('td', {"class":"alignLeft"})
len(currency_codes), len(currency_names)

(32, 32)

In [7]:
codes = [code.a.text for code in currency_codes]
names = [name.a.text for name in currency_names]

In [8]:
#Utilizar los códigos y los nombres para crear un DataFrame
#..
df.head()

,currency_code,name
0,USD,US dollar
1,JPY,Japanese yen
2,BGN,Bulgarian lev
3,CZK,Czech koruna
4,DKK,Danish krone


In [9]:
#Agregar la información sobre el Euro
df =
df.shape

(33, 2)

# API

En esta parte, se utilizará un API para obtener el tipo de cambio respecto al dolar de cada una de las monedas de las sección anterior.  
Fuente: https://exchangeratesapi.io/

In [10]:
endpoint = 

In [11]:
endpoint = endpoint.replace('\n', '')
endpoint = endpoint.replace(' ', '')
endpoint

'https://api.exchangeratesapi.io/latest?base=USD'

In [12]:
response =
response.status_code, response.reason

(200, 'OK')

In [14]:
#Convertir la rspuesta del API en un DataFrame
import json
#..
exchange_df.sample(5)

,rates,base,date
RON,4.225855,USD,2019-08-08
BGN,1.747342,USD,2019-08-08
THB,30.745109,USD,2019-08-08
RUB,65.345037,USD,2019-08-08
DKK,6.667649,USD,2019-08-08


In [15]:
exchange_df.shape

(33, 3)

____

#### Unir la información sobre el Tipo de cambio a cada moneda

In [16]:
df = 
df.head()

,currency_code,name,rates,base,date
0,USD,US dollar,1.000000,USD,2019-08-08
1,JPY,Japanese yen,105.994818,USD,2019-08-08
2,BGN,Bulgarian lev,1.747342,USD,2019-08-08
3,CZK,Czech koruna,23.040293,USD,2019-08-08
4,DKK,Danish krone,6.667649,USD,2019-08-08


____

# Database 

Los datos recolectados hasta el momento se uniran con información adicional sobre cada país para tener una base de datos final aún más completa.

In [17]:
import sqlite3

In [18]:
conn = sqlite3.connect("countries.db")

In [20]:
# Se utilizará la tabla all_countries para crear un DataFrame con la información
query = 
countries_df = 
countries_df.sample(5)

,index,country_code,country_name,country_alpha3_code,country_numeric_code,capital,country_demonym,total_area,population,idd_code,currency_code,currency_name,currency_symbol,lang_code,lang_name,cctld
85,85,GN,Guinea,GIN,324,Conakry,Guineans,245857.0,13052608,224,GNF,Guinean Franc,FG,FR,French,gn
189,189,RS,Serbia,SRB,688,Belgrade,Serbians,77474.0,8762027,381,RSD,Serbian Dinar,din,SR,Serbian,rs
130,130,LR,Liberia,LBR,430,Monrovia,Liberians,111369.0,4853516,231,LRD,Liberian Dollar,$,EN,English,lr
61,61,DZ,Algeria,DZA,12,Algiers,Algerians,2381741.0,42008054,213,DZD,Algerian Dinar,DA,AR,Arabic,dz
197,197,SG,Singapore,SGP,702,Singapore,Singaporeans,697.0,5791901,65,SGD,Singapore Dollar,$,EN,English,sg


In [21]:
#Filtrar solo los países que posean monedas manejadas por el ECB
ecb_df = 

In [22]:
#Para el caso de monedas como el Euro, que son utilizadas en múltiples países,
#solo se tomará en cuenta uno de estos en este problema
ecb_df = 

In [23]:
ecb_df.sample(5)

,index,country_code,country_name,country_alpha3_code,country_numeric_code,capital,country_demonym,total_area,population,idd_code,currency_code,currency_name,currency_symbol,lang_code,lang_name,cctld
99,99,HU,Hungary,HUN,348,Budapest,Hungarians,93028.0,9688847,36,HUF,Hungarian Forint,Ft,HU,Hungarian,hu
100,100,ID,Indonesia,IDN,360,Jakarta,Indonesians,1904569.0,266794980,62,IDR,Indonesian Rupiah,Rp,ID,Indonesian,id
47,47,CN,China,CHN,156,Beijing,Chinese,9596961.0,1415045928,86,CNY,Chinese Yuan Renminbi,¥,ZH,Chinese,cn
58,58,DK,Denmark,DNK,208,Copenhagen,Danes,43094.0,5754356,45,DKK,Danish Krone,kr.,DA,Danish,dk
157,157,MY,Malaysia,MYS,458,Kuala Lumpur,Malaysians,329847.0,32042458,60,MYR,Malaysian Ringgit,RM,MS,Malay,my


In [24]:
#Unir la información sobre países
df =

In [25]:
df

,currency_code,name,rates,base,date,index,country_code,country_name,country_alpha3_code,country_numeric_code,capital,country_demonym,total_area,population,idd_code,currency_name,currency_symbol,lang_code,lang_name,cctld
0,USD,US dollar,1.000000,USD,2019-08-08,10,AS,American Samoa,ASM,16,Pago Pago,American Samoans,199.0,55679,1684,United States Dollar,$,EN,Samoan,as
1,JPY,Japanese yen,105.994818,USD,2019-08-08,113,JP,Japan,JPN,392,Tokyo,Japanese,377915.0,127185332,81,Japanese Yen,¥,JA,Japanese,jp
2,BGN,Bulgarian lev,1.747342,USD,2019-08-08,21,BG,Bulgaria,BGR,100,Sofia,Bulgarians,110879.0,7036848,359,Bulgarian Lev,лв,GB,Bulgarian,bg
3,CZK,Czech koruna,23.040293,USD,2019-08-08,55,CZ,Czech Republic,CZE,203,Prague,Czechs,78867.0,10625250,420,Czech Koruna,Kč,CS,Czech,cz
4,DKK,Danish krone,6.667649,USD,2019-08-08,58,DK,Denmark,DNK,208,Copenhagen,Danes,43094.0,5754356,45,Danish Krone,kr.,DA,Danish,dk
5,GBP,Pound sterling,0.822970,USD,2019-08-08,76,GB,United Kingdom,GBR,826,London,British,243610.0,66573504,44,British Pound,£,EN,English,gb
6,HUF,Hungarian forint,290.431520,USD,2019-08-08,99,HU,Hungary,HUN,348,Budapest,Hungarians,93028.0,9688847,36,Hungarian Forint,Ft,HU,Hungarian,hu
7,PLN,Polish zloty,3.861967,USD,2019-08-08,178,PL,Poland,POL,616,Warsaw,Poles,312685.0,38104832,48,Polish Zloty,zł,PL,Polish,pl
8,RON,Romanian leu,4.225855,USD,2019-08-08,188,RO,Romania,ROU,642,Bucharest,Romanians,238391.0,19580634,40,Romanian New Lei,RON,RO,Romanian,ro
9,SEK,Swedish krona,9.606897,USD,2019-08-08,196,SE,Sweden,SWE,752,Stockholm,Swedes,450295.0,9982709,46,Swedish Krona,kr,SV,Swedish,se
